In [14]:
import arcpy
import pandas as pd
import os

# Path to your enterprise geodatabase connection (.sde file)
sde_connection = r"F:\GIS\DB_CONNECT\Tabular.sde"

# Set workspace
arcpy.env.workspace = sde_connection

def clean_name(name):
    """Remove schema prefix (like sde.) from table/fc names."""
    if "." in name:
        return name.split(".")[-1]
    return name

items = []

# --- Tables ---
for tbl in arcpy.ListTables() or []:
    try:
        desc = arcpy.Describe(tbl)
        items.append({
            "Name": clean_name(tbl),
            "Schema": tbl.split(".")[0] if "." in tbl else None,
            "Type": "Table",
            "Dataset": None,
            "DataType": desc.dataType,
            "Path": desc.catalogPath
        })
    except Exception as e:
        items.append({
            "Name": clean_name(tbl),
            "Schema": tbl.split(".")[0] if "." in tbl else None,
            "Type": "Table",
            "Dataset": None,
            "DataType": "Error",
            "Path": str(e)
        })

# --- Feature Classes in root ---
for fc in arcpy.ListFeatureClasses() or []:
    try:
        desc = arcpy.Describe(fc)
        items.append({
            "Name": clean_name(fc),
            "Schema": fc.split(".")[0] if "." in fc else None,
            "Type": "FeatureClass",
            "Dataset": None,
            "DataType": desc.dataType,
            "ShapeType": getattr(desc, "shapeType", None),
            "Path": desc.catalogPath
        })
    except Exception as e:
        items.append({
            "Name": clean_name(fc),
            "Schema": fc.split(".")[0] if "." in fc else None,
            "Type": "FeatureClass",
            "Dataset": None,
            "DataType": "Error",
            "ShapeType": None,
            "Path": str(e)
        })

# --- Feature Classes inside Datasets ---
for ds in arcpy.ListDatasets("", "Feature") or []:
    arcpy.env.workspace = os.path.join(sde_connection, ds)
    for fc in arcpy.ListFeatureClasses() or []:
        try:
            desc = arcpy.Describe(fc)
            items.append({
                "Name": clean_name(fc),
                "Schema": fc.split(".")[0] if "." in fc else None,
                "Type": "FeatureClass",
                "Dataset": clean_name(ds),
                "DataType": desc.dataType,
                "ShapeType": getattr(desc, "shapeType", None),
                "Path": desc.catalogPath
            })
        except Exception as e:
            items.append({
                "Name": clean_name(fc),
                "Schema": fc.split(".")[0] if "." in fc else None,
                "Type": "FeatureClass",
                "Dataset": clean_name(ds),
                "DataType": "Error",
                "ShapeType": None,
                "Path": str(e)
            })

# Reset workspace
arcpy.env.workspace = sde_connection

# --- DataFrame ---
df = pd.DataFrame(items)

print(df)

# Save to CSV
df.to_csv("gdb_contents.csv", index=False)


                           Name Schema   Type Dataset DataType  \
0    tahoe_yellow_cress_surveys    SDE  Table    None    Table   
1           limebike_pilot_2017    SDE  Table    None    Table   
2           secchi_measurements    SDE  Table    None    Table   
3                   LandUseCode    SDE  Table    None    Table   
4                 LocalPlan_URL    SDE  Table    None    Table   
..                          ...    ...    ...     ...      ...   
130       Accela_Record_Details    sde  Table    None    Error   
131              Accela_Parcels    dbo  Table    None    Table   
132       Accela_Record_Details    dbo  Table    None    Table   
133    bike_ped_counter_tabular    sde  Table    None    Error   
134           Parcel_APN_NewOld    sde  Table    None    Table   

                                                  Path  
0    F:\GIS\DB_CONNECT\Tabular.sde\SDE.tahoe_yellow...  
1    F:\GIS\DB_CONNECT\Tabular.sde\SDE.limebike_pil...  
2    F:\GIS\DB_CONNECT\Tabular.sde\S

In [6]:
df_collect = df.copy()


In [8]:
df_vector = df.copy()

In [15]:
df_tabular = df.copy()

In [11]:
df_collect['database']='collect'
df_vector['database']='vector'

In [16]:
df_tabular['database']='tabular'

In [17]:
df_combined = pd.concat([df_vector,df_collect,df_tabular])

In [18]:
df_combined.to_csv('combined2.csv')